In [ ]:
pip install scikeras

In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from keras.callbacks import EarlyStopping



In [ ]:

df = pd.DataFrame({
    "soil_moisture": [0.10, 0.15, 0.20, 0.25, 0.40, 0.60, 0.35, 0.18,
                      0.45, 0.05, 0.80, 0.27, 0.55, 0.70, 0.12, 0.30],
    "temperature_c": [34, 30, 26, 22, 28, 30, 19, 22,
                      35, 24, 33, 33, 21, 25, 20, 29],
    "sunlight_hours": [9, 8, 7, 4, 8, 10, 3, 10,
                       12, 5, 9, 11, 2, 6, 1, 9],
    "needs_water": [1, 1, 1, 0, 0, 0, 0, 1,
                    0, 1, 0, 1, 0, 0, 1, 1]
})

In [ ]:
df

,soil_moisture,temperature_c,sunlight_hours,needs_water
0,0.10,34,9,1
1,0.15,30,8,1
2,0.20,26,7,1
3,0.25,22,4,0
4,0.40,28,8,0
5,0.60,30,10,0
6,0.35,19,3,0
7,0.18,22,10,1
8,0.45,35,12,0
9,0.05,24,5,1


In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num',MinMaxScaler(),['soil_moisture','temperature_c','sunlight_hours'])
    ]
)

In [ ]:
X,y=df.drop('needs_water',axis=1),df["needs_water"]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train.shape[0]

12

In [ ]:
model=keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(
    X_train,y_train,
    validation_data=(X_test.values,y_test.values),
    epochs=100,
    batch_size=4,
    verbose=1
)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.5312 - loss: 1.8568 - val_accuracy: 0.2500 - val_loss: 3.5607
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7917 - loss: 0.7833 - val_accuracy: 0.0000e+00 - val_loss: 2.0119
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.6354 - loss: 1.0351 - val_accuracy: 0.2500 - val_loss: 4.5071
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6979 - loss: 0.8692 - val_accuracy: 0.5000 - val_loss: 1.8215
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5312 - loss: 1.6401 - val_accuracy: 0.7500 - val_loss: 2.2588
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5000 - loss: 2.3029 - val_accuracy: 0.5000 - val_loss: 1.6774
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4375 - loss: 2.1352 - val_accuracy: 0.5000 - val_loss: 1.4724
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4583 - loss: 1.6076 - val_accuracy: 0.2500 - val_

In [ ]:
def make_model():
  model=keras.Sequential([
      layers.Input(shape=(X_train.shape[1],)),
      layers.Dense(8,activation='relu'),
      layers.Dense(1,activation='sigmoid')
  ])
  model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
  return model
clf=KerasClassifier(model=make_model,epochs=100,batch_size=4, verbose=1)

In [ ]:
preprocessor=ColumnTransformer(
    transformers=[
        ('num',MinMaxScaler(),['soil_moisture','temperature_c','sunlight_hours'])
    ]
)

In [ ]:
pipeline=Pipeline([
    ("preprocessor",preprocessor),
    ("clf",clf)
])

In [ ]:
es=EarlyStopping(
    monitor="val_accuracy",
    patience=10,
    restore_best_weights=True
)


In [ ]:
history=pipeline.fit(
    X_train,y_train,
    # clf__validation_split=0.2,
    clf__callbacks=[es],
    clf__verbose=1
)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.5521 - loss: 0.8146 
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3646 - loss: 0.9062 
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3958 - loss: 0.8974
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3646 - loss: 0.8626
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5521 - loss: 0.7766
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3021 - loss: 0.9274    
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4583 - loss: 0.8567 
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4271 - loss: 0.7927
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3646 - loss: 0.8504
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3229 - loss: 0.8487
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2917 - loss: 0.8503
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.26

In [ ]:
history

,steps,"[('preprocessor', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
test_acc = pipeline.score(X_test, y_test)
print("Final test accuracy:", test_acc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Final test accuracy: 0.25


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
hist = pipeline.named_steps["clf"].model.history.history

plt.plot(hist["loss"], label="Train Loss")
plt.plot(hist["val_loss"], label="Val Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.plot(hist["accuracy"], label="Train Accuracy")
plt.plot(hist["val_accuracy"], label="Val Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

AttributeError: 'function' object has no attribute 'history'